#Pesquisa

O SITE

Como um amante de series eu sempre quis estudalas mais a fundo, principalmente sobre as suas recepções pelo público. Mas para isso eu precisava coletar dados sobre elas de uma fonte imparcial e de prestigio. Com isso em mente o IMDB foi uma escolha certeira se baseanda apenas em fatores estatisticos. Assim esse trabalho de web scraping surgiu

OS PROBLEMAS

Uma varredura analogica no site revelou alguns problemas para a coleta.
A maioria das listas de popularidade incluem series que ainda não foram exibidas faltando assim muitas informassoes delas. Para reesolver isso foi escolhida uma lista baseada no numero de votos dos usuários para conferir um nivel de engajamento em cada seria analizada. Algo mais comum de acontecer com eries que já tiveram o seu lançamento. 

Por questões envolvendo a formulação web do site alguns dados como o elenco principal e a media de tempo de cada episodeo tiveram que ficar de fora da coleta.

OS DADOS

Por se uma pesquisa estatistica sobre a avaliação dos usuários foram escolhidos elementos que abordem o contexto de cada serie e que representão o nivel da sua "popularidade e engajamento por parte do publico.Dito isso os elementos coletados foram 


Posição: Que representa em qual o hank de numeros de avaliações a serie se encontra.

Nome: O nome da serie.

Ano: Anos respectivos ao começo e finalização da serie.

Genero: Generos cinematograficos nos quais a serie aborda.

Votos: Numero total de votos.


# Importando as bibliotecas

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
import urllib.request as urllib_request
from urllib.request import Request, urlopen

In [3]:
#Por uma questão de formatação das paginas será preciso de um tratamento expecifico do nome antes do scraping

def nome(serie):
  lines = serie.find('h3', {'class':'lister-item-header'})
  item = lines.text.split('.')[1]
  card['name'] = item.split('(')[0]
  card['name'] = item.split('\n\n\n')[0]

In [4]:
# Para fazer o scraping nas paginas do imdb será preciso de uma lista e um dicionario
cards = []
card = {}

In [5]:
#A variavel i vai serr umportante para passar as paginas
i= 1

In [6]:



# Estabelecendo o fim da coleta (500 series)
while i < 500:
  #definindo o começo da coleta
  response = urlopen('https://www.imdb.com/search/title/?title_type=tv_series&sort=num_votes,desc&start={}&ref_=adv_nxt'.format(i))
  html = response.read()
  html = html.decode('utf-8')
  ## O BeautifulSoup sendo usado para permitir uma pesquisa dentro da pagina
  soup = BeautifulSoup(html, 'html.parser')

#cortando s pagina de busca para criar uma lista com cada serie a ser coletada
  series = soup.find('div', {'class': 'lister-list'}).findAll('div', class_= 'lister-item mode-advanced')
  #criando uma rotina para a coleta
  for serie in series:
    card = {}
    #definindo os elementos a serem coletados
    card['position'] = serie.find('span', {'class':'lister-item-index unbold text-primary'}).get_text().split('.')[0]
    nome(serie)
    card['ano'] = serie.find('span', {'class':'lister-item-year text-muted unbold'}).get_text()
    card['genre'] = serie.find('span', {'class':'genre'}).get_text() 
    card['note'] = serie.find('div', {'class':'inline-block ratings-imdb-rating'}).get_text()
    card['votos'] = serie.find('span', {'name':'nv'}).get_text() 
    cards.append(card)
  #Para definir uma nova pagina é preciso adicionar 50 ao numera da pagina anterior. Por isso a adição de +50 ao i
  i+=50 
#Quardando todos os elementos coletados em um DataFrame
dataset = pd.DataFrame(cards)

In [7]:
#Um pequeno tratamento nos dados coletados
lines = range(dataset.shape[0] )
for line in lines:
  dataset['name'][line] = dataset['name'][line].split('\n')[1]
  dataset['genre'][line] = dataset['genre'][line].split('\n')[1]
  dataset['note'][line] = dataset['note'][line].split('\n')[2]

In [8]:
dataset

,position,name,ano,genre,note,votos
0,1,Game of Thrones,(2011–2019),"Action, Adventure, Drama",9.2,"1,997,704"
1,2,Breaking Bad,(2008–2013),"Crime, Drama, Thriller",9.5,"1,750,746"
2,3,Stranger Things,(2016– ),"Drama, Fantasy, Horror",8.7,"1,044,155"
3,4,Friends,(1994–2004),"Comedy, Romance",8.9,"957,106"
4,5,The Walking Dead,(2010–2022),"Drama, Horror, Thriller",8.2,"950,680"
...,...,...,...,...,...,...
495,496,Yellowjackets,(2021– ),"Drama, Horror, Mystery",7.9,"39,255"
496,497,Empire,(2015–2020),"Drama, Music",7.3,"39,240"
497,498,Drake & Josh,(2004–2007),"Comedy, Family",7.7,"39,223"
498,499,Impractical Jokers,(2011– ),"Comedy, Reality-TV",8.6,"39,133"


In [9]:
# importando os dados
dataset.to_csv('series_+votadas.csv')
